### Import All Libraries

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import random
import os, sys
import itertools, time
from Model import get_Model
from parameter import letters
from keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg



Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


Using TensorFlow backend.


In [2]:
import tensorflow as tf
tf.__version__



'2.3.0-rc0'

### GPU Settings ( do not run in no GPU env )

In [ ]:
tf.debugging.set_log_device_placement(True)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
start_time = time.time()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

### line detection

In [3]:

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    #print(img.shape)
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def draw_circle(img,lines, color=[0, 0, 255]):
    for line in lines:
        cv2.circle(img,(line[0],line[1]), 2, color, -1)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_arr = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    #draw_lines(line_arr, lines)
    return lines

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def Collect_points(lines):

    # reshape [:4] to [:2]
    interp = lines.reshape(lines.shape[0]*2,2)
    # interpolation & collecting points for RANSAC
    for line in lines:
        if np.abs(line[3]-line[1]) > 5:
            tmp = np.abs(line[3]-line[1])
            a = line[0] ; b = line[1] ; c = line[2] ; d = line[3]
            slope = (line[2]-line[0])/(line[3]-line[1]) 
            for m in range(0,tmp,5):
                if slope>0:
                    new_point = np.array([[int(a+m*slope),int(b+m)]])
                    interp = np.concatenate((interp,new_point),axis = 0)
                elif slope<0:
                    new_point = np.array([[int(a-m*slope),int(b-m)]])
                    interp = np.concatenate((interp,new_point),axis = 0)                
    return interp

def get_random_samples(lines):
    one = random.choice(lines)
    two = random.choice(lines)
    if(two[0]==one[0]): # extract again if values are overlapped
        while two[0]==one[0]:
            two = random.choice(lines)
    one, two = one.reshape(1,2), two.reshape(1,2)
    three = np.concatenate((one,two),axis=1)
    three = three.squeeze()
    return three

def compute_model_parameter(line):
    # y = mx+n
    m = (line[3] - line[1])/(line[2] - line[0])
    n = line[1] - m*line[0]
    # ax+by+c = 0
    a, b, c = m, -1, n
    par = np.array([a,b,c])
    return par

def compute_distance(par, point):
    # distance between line & point
    
    return np.abs(par[0]*point[:,0]+par[1]*point[:,1]+par[2])/np.sqrt(par[0]**2+par[1]**2)

def model_verification(par, lines):
    # calculate distance
    distance = compute_distance(par,lines)
    # total sum of distance between random line and sample points    
    sum_dist = distance.sum(axis=0)
    # average
    avg_dist = sum_dist/len(lines)
    
    return avg_dist

def draw_extrapolate_line(img, par,color=(0,0,255), thickness = 2):

    x1, y1 = int(-par[1]/par[0]*img.shape[0]-par[2]/par[0]), int(img.shape[0])
    x2, y2 = int(-par[1]/par[0]*(img.shape[0]/2+100)-par[2]/par[0]), int(img.shape[0]/2+100)
    cv2.line(img, (x1 , y1), (x2, y2), color, thickness)
    return img

def get_fitline(img, f_lines):

    rows,cols = img.shape[:2]
    output = cv2.fitLine(f_lines,cv2.DIST_L2,0, 0.01, 0.01)
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    x1, y1 = int(((img.shape[0]-1)-y)/vy*vx + x) , img.shape[0]-1
    x2, y2 = int(((img.shape[0]/2+100)-y)/vy*vx + x) , int(img.shape[0]/2+100)
    result = [x1,y1,x2,y2]

    return result

def draw_fitline(img, result_l,result_r, color=(255,0,255), thickness = 10):
    # draw fitting line
    lane = np.zeros_like(img)
    cv2.line(lane, (int(result_l[0]) , int(result_l[1])), (int(result_l[2]), int(result_l[3])), color, thickness)
    cv2.line(lane, (int(result_r[0]) , int(result_r[1])), (int(result_r[2]), int(result_r[3])), color, thickness)
    # add original image & extracted lane lines
    final = weighted_img(lane, img, 1,0.5)  
    return final

def erase_outliers(par, lines):
    # distance between best line and sample points
    distance = compute_distance(par,lines)

    #filtered_dist = distance[distance<15]
    filtered_lines = lines[distance<13,:]
    return filtered_lines

def smoothing(lines, pre_frame):
    # collect frames & print average line
    lines = np.squeeze(lines)
    avg_line = np.array([0,0,0,0])
    
    for ii,line in enumerate(reversed(lines)):
        if ii == pre_frame:
            break
        avg_line += line
    avg_line = avg_line / pre_frame

    return avg_line

def ransac_line_fitting(img, lines, min=100):
    global fit_result, l_fit_result, r_fit_result
    best_line = np.array([0,0,0])
    if(len(lines)!=0):                
        for i in range(30):           
            sample = get_random_samples(lines)
            parameter = compute_model_parameter(sample)
            cost = model_verification(parameter, lines)                        
            if cost < min: # update best_line
                min = cost
                best_line = parameter
            if min < 3: break
        # erase outliers based on best line
        filtered_lines = erase_outliers(best_line, lines)
        fit_result = get_fitline(img, filtered_lines)
    else:
        if (fit_result[3]-fit_result[1])/(fit_result[2]-fit_result[0]) < 0:
            l_fit_result = fit_result
            return l_fit_result
        else:
            r_fit_result = fit_result
            return r_fit_result
        
    try:
        if (fit_result[3]-fit_result[1])/(fit_result[2]-fit_result[0]) < 0:
            l_fit_result = fit_result
            return l_fit_result
        else:
            r_fit_result = fit_result
            return r_fit_result
    except:
        r_fit_result = fit_result
        return r_fit_result

def perspective_transform(img):
    # [x,y] 좌표점을 4x2의 행렬로 작성
    # 좌표점은 좌상->좌하->우상->우하
    pts1 = np.float32([[0,0],[350,200],[200,400],[350,400]])

    # 좌표의 이동점
    pts2 = np.float32([[0,0],[600,0],[0,800],[600,800]])

    # pts1의 좌표에 표시. perspective 변환 후 이동 점 확인.
    cv2.circle(img, (504,1003), 20, (255,0,0),-1)
    cv2.circle(img, (243,1524), 20, (0,255,0),-1)
    cv2.circle(img, (1000,1000), 20, (0,0,255),-1)
    cv2.circle(img, (1280,1685), 20, (0,0,0),-1)

    M = cv2.getPerspectiveTransform(pts1, pts2)

    dst = cv2.warpPerspective(img, M, (1100,1100))

    #plt.subplot(121),plt.imshow(img),plt.title('image')
    #plt.subplot(122),plt.imshow(dst),plt.title('Perspective')
    #plt.show()
    return dst
    
def detect_lanes_img(img):

    height, width = img.shape[:2]
    pers_img = perspective_transform(img)

    # Set ROI
    vertices = np.array([[(50,height),(width/2-45, height/2+60), (width/2+45, height/2+60), (width-50,height)]], dtype=np.int32)
    ROI_img = region_of_interest(pers_img, vertices)
    
    # Convert to grayimage
    #g_img = grayscale(img)
       
    # Apply gaussian filter
    blur_img = gaussian_blur(img, 3)
        
    # Apply Canny edge transform
    canny_img = canny(blur_img, 70, 210)
    # to except contours of ROI image
    vertices2 = np.array([[(52,height),(width/2-43, height/2+62), (width/2+43, height/2+62), (width-52,height)]], dtype=np.int32)
    canny_img = region_of_interest(canny_img, vertices2)

    # Perform hough transform
    # Get first candidates for real lane lines  
    line_arr = hough_lines(canny_img, 1, 1 * np.pi/180, 30, 10, 20)
    
    #draw_lines(img, line_arr, thickness=2)

    line_arr = np.squeeze(line_arr)
    
    try:
        # Get slope degree to separate 2 group (+ slope , - slope)
        slope_degree = (np.arctan2(line_arr[:,1] - line_arr[:,3], line_arr[:,0] - line_arr[:,2]) * 180) / np.pi

        # ignore horizontal slope lines
        line_arr = line_arr[np.abs(slope_degree)<160]
        slope_degree = slope_degree[np.abs(slope_degree)<160]
        # ignore vertical slope lines
        line_arr = line_arr[np.abs(slope_degree)>95]
        slope_degree = slope_degree[np.abs(slope_degree)>95]
        L_lines, R_lines = line_arr[(slope_degree>0),:], line_arr[(slope_degree<0),:]
        #print(line_arr.shape,'  ',L_lines.shape,'  ',R_lines.shape)
    
        # interpolation & collecting points for RANSAC
        L_interp = Collect_points(L_lines)
        R_interp = Collect_points(R_lines)

        #draw_circle(img,L_interp,(255,255,0))
        #draw_circle(img,R_interp,(0,255,255))

        # erase outliers based on best line
        left_fit_line = ransac_line_fitting(img, L_interp)
        right_fit_line = ransac_line_fitting(img, R_interp)

        # smoothing by using previous frames
        L_lane.append(left_fit_line), R_lane.append(right_fit_line)

        if len(L_lane) > 10:
            left_fit_line = smoothing(L_lane, 10)    
        if len(R_lane) > 10:
            right_fit_line = smoothing(R_lane, 10)
        final = draw_fitline(img, left_fit_line, right_fit_line)
    except:
        final = img

    return final


### Plate Detection by RCNN Prediction models

In [4]:

''' plate Prediction '''
K.set_learning_phase(0)

Region = {"A": "서울 ", "B": "경기 ", "C": "인천 ", "D": "강원 ", "E": "충남 ", "F": "대전 ",
          "G": "충북 ", "H": "부산 ", "I": "울산 ", "J": "대구 ", "K": "경북 ", "L": "경남 ",
          "M": "전남 ", "N": "광주 ", "O": "전북 ", "P": "제주 "}
Hangul = {"dk": "아", "dj": "어", "dh": "오", "dn": "우", "qk": "바", "qj": "버", "qh": "보", "qn": "부",
          "ek": "다", "ej": "더", "eh": "도", "en": "두", "rk": "가", "rj": "거", "rh": "고", "rn": "구",
          "wk": "자", "wj": "저", "wh": "조", "wn": "주", "ak": "마", "aj": "머", "ah": "모", "an": "무",
          "sk": "나", "sj": "너", "sh": "노", "sn": "누", "fk": "라", "fj": "러", "fh": "로", "fn": "루",
          "tk": "사", "tj": "서", "th": "소", "tn": "수", "gj": "허"}

def decode_label(out):
    # out : (1, 32, 42)
    out_best = list(np.argmax(out[0, 2:], axis=1))  # get max index -> len = 32
    out_best = [k for k, g in itertools.groupby(out_best)]  # remove overlap value
    outstr = ''
    for i in out_best:
        if i < len(letters):
            outstr += letters[i]
            '''
            if letters[i] != "X":
                outstr += letters[i]
            '''
    return outstr


def label_to_hangul(label):  # eng -> hangul
    #print("label in prediction:",label)
    
    if len(label)==10 and label[9] == "X":
        region = label[0]
        two_num = label[1:3]
        hangul = label[3:5]
        four_num = label[5:9]
        try:
            region = Region[region] if region != 'Z' else ''
        except:
            pass
        try:
            hangul = Hangul[hangul]
        except:
            pass
        return region + two_num + hangul + four_num
    else:
        #print("label:",label)
        region = label[0]
        three_num = label[1:4]
        hangul = label[4:6]
        four_num = label[6:]
        if len(four_num)==5:
            four_num = label[6:10]
        try:
            region = Region[region] if region != 'Z' else ''
        except:
            pass
        try:
            hangul = Hangul[hangul]
        except:
            pass
        return region + three_num + hangul + four_num

model = get_Model(training=False)

try:
    model.load_weights("LSTM+BN9--90--3.020.hdf5")
    print("...Previous weight data...")
except:
    raise Exception("No weight file!")

total = 0
acc = 0
letter_total = 0
letter_acc = 0
start = time.time()
    

...Previous weight data...


### Object Detection model

### Show the CV2 Windows (실행부분)

In [5]:
cap = cv2.VideoCapture(0) # 웹캠 가져오기
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) # 또는 cap.get(3)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # 또는 cap.get(4)
fps = cap.get(cv2.CAP_PROP_FPS) # 또는 cap.get(5)
print('프레임 너비: %d, 프레임 높이: %d, 초당 프레임 수: %d' %(width, height, fps))
cap.set(3,1280)
cap.set(4,1024)

width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) # 또는 cap.get(3)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # 또는 cap.get(4)
print('프레임 너비: %d, 프레임 높이: %d, 초당 프레임 수: %d' %(width, height, fps))

fit_result, l_fit_result, r_fit_result, L_lane, R_lane = [], [], [], [], []

# Define the codec and create VideoWriter object
#fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use the lower case
#out = cv2.VideoWriter('output.mp4', fourcc, 20.0, ( 960, 540 ))

프레임 너비: 640, 프레임 높이: 480, 초당 프레임 수: 60
프레임 너비: 1280, 프레임 높이: 720, 초당 프레임 수: 60


In [6]:
from time import sleep
print("capture open?:",cap.isOpened())
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret: # ret = true or False
        result = detect_lanes_img(frame)
        #rint("loaded line detection model")

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 웹캠 이미지 grayscale 변환
        
        frame2 = frame
        img_pred = frame2.astype(np.float32)
        img_pred = cv2.resize(img_pred, (128, 64))
        img_pred = (img_pred / 255.0) * 2.0 - 1.0
        img_pred = img_pred.T
        img_pred = np.expand_dims(img_pred, axis=-1)
        img_pred = np.expand_dims(img_pred, axis=0)
        
        net_out_value = model.predict(img_pred)
        #rint("Predicting license plate")
        pred_texts = decode_label(net_out_value)
        #print("Predicted texts:", pred_texts)
        cv2.rectangle(result, (0,0), (150, 30), (0,0,0), -1)
        cv2.putText(result, pred_texts, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255),2)
        
        
        cv2.imshow('result',result)
    else:
        break
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

capture open?: True
